### Install and import necessary packages

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# Tools for scaling data, PCA, and standard datasets
from sklearn import preprocessing, decomposition, datasets
import numpy as np
import pandas as pd

## Load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)
!ls '/content/drive/MyDrive/eth_CIL_exercise/data/logreg_input'
valm1 = pd.read_csv('/content/drive/MyDrive/eth_CIL_exercise/data/logreg_input/pred_val_1-cardiff.csv',names= ['p10', 'p11'])
valm2 = pd.read_csv('/content/drive/MyDrive/eth_CIL_exercise/data/logreg_input/pred_val_1-bertweet.csv',names= ['p20', 'p21'])
valm3 = pd.read_csv('/content/drive/MyDrive/eth_CIL_exercise/data/logreg_input/pred_val_1-xlnet.csv',names= ['p30', 'p31'])
testm1 = pd.read_csv('/content/drive/MyDrive/eth_CIL_exercise/data/logreg_input/pred_test-prob_1-cardiff.csv',names= ['p10', 'p11'])
testm2 = pd.read_csv('/content/drive/MyDrive/eth_CIL_exercise/data/logreg_input/pred_test-prob_1-bertweet.csv',names= ['p20', 'p21'])
testm3 = pd.read_csv('/content/drive/MyDrive/eth_CIL_exercise/data/logreg_input/pred_test-prob_1-xlnet.csv',names= ['p30', 'p31'])

Mounted at /content/drive
gold.csv		       pred_test-prob_1-xlnet.csv  pred_val_1-xlnet.csv
pred_test-prob_1-bertweet.csv  pred_val_1-bertweet.csv
pred_test-prob_1-cardiff.csv   pred_val_1-cardiff.csv


In [ ]:
#COMPARE: df_val['sentiment'] vs gold, are they the same function set
df_val = pd.read_csv('/content/drive/MyDrive/eth_CIL_exercise/data/clean_val.csv')
gold = pd.read_csv('/content/drive/MyDrive/eth_CIL_exercise/data/logreg_input/gold.csv',names= ['sentiment'])
cm = confusion_matrix(df_val['sentiment'] , gold) #so gold = df_val['sentiment']
cm

array([[249742,      0],
       [     0, 250258]])

In [ ]:
#merge data
X_trainf = pd.concat([valm1, valm2,valm3], axis=1)
y_trainf = gold
X_test = pd.concat( [testm1,testm2,testm3], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
#split data
X_train, X_val, y_train, y_val = train_test_split(X_trainf, y_trainf, test_size=0.5, random_state=68)
X_train, y_train = shuffle(X_train, y_train, random_state=68) # shuffle the training data
X_val, y_val = shuffle(X_val, y_val, random_state=68)

In [ ]:
#scale the data
standard_scaler = preprocessing.StandardScaler()
X_train_standard = standard_scaler.fit_transform(X_train)
X_val_standard = standard_scaler.transform(X_val)
X_test_standard = standard_scaler.transform(X_test)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)

(250000, 6)
(250000, 1)
(250000, 6)
(250000, 1)
(10000, 6)


## Finetuning LogReg model

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, validation_curve, learning_curve
# The LogReg learning algorithm
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state = 88,
                            penalty = 'l2')
logreg.fit(X_train_standard, y_train)
logreg.score(X_train_standard, y_train), logreg.score(X_val_standard, y_val)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.9181, 0.91708)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
logregcv = LogisticRegressionCV(cv=4, random_state=99)
logregcv.fit(X_train_standard, y_train)
logregcv.score(X_train_standard, y_train), logregcv.score(X_val_standard, y_val)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.918092, 0.91708)

In [ ]:
#predict
pred_lrcv = logregcv.predict(X_test)
pred_lr = logreg.predict(X_test_standard)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
pred_test = pd.DataFrame({'Prediction': pred_lr})
pred_test = pd.DataFrame({'Id': pred_test.index + 1 ,'Prediction': pred_lr}) #add columns Id as sample_submission file
pred_test.loc[pred_test['Prediction'] < 1, 'Prediction'] = -1

In [ ]:
from google.colab import files
pred_test.to_csv('pred_test_lr.csv',index=False)
files.download('pred_test_lr.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## knn


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=15)
y_pred = neigh.fit(X_train_standard, y_train).predict(X_val_standard)
accuracy_score(y_val, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.9122333333333333

In [ ]:
pred_knn = neigh.predict(X_test)
pred_test = pd.DataFrame({'Prediction': pred_knn})
pred_test = pd.DataFrame({'Id': pred_test.index + 1 ,'Prediction': pred_knn}) #add columns Id as sample_submission file
pred_test.loc[pred_test['Prediction'] < 1, 'Prediction'] = -1
from google.colab import files
pred_test.to_csv('pred_test_knn.csv',index=False)
files.download('pred_test_knn.csv')

## Naive BAyes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train_standard, y_train).predict(X_val_standard)
accuracy_score(y_val, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9085866666666667